In [ ]:
#hide
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
#hide
from fastai2.vision.all import *

In [ ]:
#hide
class ApplyPILFilter(RandTransform): pass
path_model = '/Users/rahulsomani/Desktop/shot-lighting-cast/fastai2-110-epoch-model.pkl'

learn = load_learner(path_model);

/Users/rahulsomani/anaconda3/lib/python3.7/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/rahulsomani/anaconda3/lib/python3.7/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.activation.ReLU' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/rahulsomani/anaconda3/lib/python3.7/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by 

You need to change fastai's `Flatten` to `nn.Flatten()` for CoreML compatibility

In [ ]:
x = torch.rand(1,3,224,224)
out1 = learn.model(x)
learn.model[1][1] = nn.Flatten()
out2 = learn.model(x)

torch.equal(out1,out2)

True

### Pre-Processing & Predicting in PyTorch

In [ ]:
import torchvision.transforms.functional as TTF
import torch.nn as nn
to_cuda = lambda x: x.cuda() if torch.cuda.is_available() else x

In [ ]:
imagenet_stats

([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

In [ ]:
PathLike = Union[str,Path]

def preprocess_one(fname:PathLike):
    x = open_image(fname)
    x = TTF.to_tensor(x)
    x = TTF.normalize(x, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    x = to_cuda(x)
    x = x.unsqueeze(0)
    return x

def preprocess_batch(fnames:Union[PathLike,Collection]):
    batch = [preprocess_one(f) for f in fnames]
    return torch.cat(batch)    

In [ ]:
path_imgs = Path('/Users/rahulsomani/Desktop/lighting-cast/')
files = get_image_files(path_imgs)
f = files[120]

In [ ]:
from pprint import pprint
import PIL

open_image = lambda f,size=(224,224): PIL.Image.open(f).convert('RGB').resize(size, PIL.Image.BILINEAR)

In [ ]:
x = preprocess_one(f)
torch_pred = nn.Softmax(-1)(learn.model(x))
torch_pred

tensor([[0.7403, 0.2597]], grad_fn=<SoftmaxBackward>)

### Export to ONNX

In [ ]:
import onnx
import onnx.utils
from onnx import optimizer

In [ ]:
def torch_to_onnx(model:nn.Module,
                  activation:nn.Module=None,
                  save_path:str     = '../exported-models/',
                  model_fname:str   = 'onnx-model',
                  input_shape:tuple = (1,3,224,224),
                  input_name:str    = 'input_image',
                  output_names:Union[str,list] = 'output',
                  **export_args) -> None:
    save_path = Path(save_path)
    if isinstance(output_names, str): output_names = [output_names]
    if activation: model = nn.Sequential(*[model, activation])
    model.eval()
    x = torch.randn(input_shape, requires_grad=True)
    x = x.cuda() if torch.cuda.is_available() else x
    model(x)
    dynamic_batch = {0: 'batch'}
    dynamic_axes  = {input_name : dynamic_batch}
    for out in output_names: dynamic_axes[out] = dynamic_batch
    torch.onnx._export(model, x, f"{save_path/model_fname}.onnx",
                       export_params=True, verbose=False,
                       input_names=[input_name], output_names=output_names,
                       dynamic_axes=dynamic_axes, keep_initializers_as_inputs=True,
                       **export_args)
    print(f"Loading, polishing, and optimising exported model from {save_path/model_fname}.onnx")
    onnx_model = onnx.load(f'{save_path/model_fname}.onnx')
    model = onnx.utils.polish_model(onnx_model)
    #onnx.checker.check_model(model)

    # removing unused parts of the model
    passes = ["extract_constant_to_initializer", "eliminate_unused_initializer"]
    optimized_model = optimizer.optimize(onnx_model, passes)

    onnx.save(optimized_model, f'{save_path/model_fname}.onnx')
    print('Done')

In [ ]:
torch_to_onnx(model = learn.model,
              activation   = nn.Softmax(-1),
              save_path    = '/Users/rahulsomani/Desktop/',
              model_fname  = 'lighting-cast',
              output_names = 'lighting-cast')

Loading, polishing, and optimising exported model from /Users/rahulsomani/Desktop/lighting-cast.onnx
Done


### ONNX Inference

In [ ]:
from onnxruntime import InferenceSession

In [ ]:
def torch_to_numpy(tensor): return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [ ]:
onnx_model_path = '/Users/rahulsomani/Desktop/lighting-cast.onnx'
session = InferenceSession(onnx_model_path)

In [ ]:
x = {session.get_inputs()[0].name:
     torch_to_numpy(preprocess_one(f))}

preds_onnx = session.run(None, x)
preds_onnx

[array([[0.74026996, 0.25973007]], dtype=float32)]

#### Batch vs Single Image Prediction Comparison

In [ ]:
%%timeit
x = {session.get_inputs()[0].name:
     torch_to_numpy(preprocess_batch(files[:10]))}

preds_onnx = session.run(None, x)
#preds_onnx

403 ms ± 74.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit
for f in files[:10]:
    x = {session.get_inputs()[0].name:
         torch_to_numpy(preprocess_one(f))}
    preds_onnx = session.run(None, x)

340 ms ± 16.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Export to TensorFlow

https://github.com/onnx/onnx-tensorflow/blob/master/example/onnx_to_tf.py

In [ ]:
import onnx_tf

In [ ]:
from onnx_tf.backend import prepare

onnx_model = onnx.load(onnx_model_path)
output = 

### Export to CoreML

In [ ]:
imagenet_stats

([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

In [ ]:
1.0 / (255.0 * 0.226)

0.01735207357279195

In [ ]:
import copy
import coremltools
import onnx_coreml
import os

from onnx_coreml import convert

In [ ]:
print(f"coremltools version: {coremltools.__version__}")
print(f"onnx-coreml version: {onnx_coreml.__version__}")

coremltools version: 3.3
onnx-coreml version: 1.3


In [ ]:
imagenet_stats

([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

In [ ]:
preprocessing_args = dict(
    image_scale = 1/255.,
    red_bias    = -0.485,
    green_bias  = -0.456,
    blue_bias   = -0.406,
    is_bgr      = False
)

red_sdev, green_sdev, blue_sdev = imagenet_stats[1]

coreml_model = convert(
    model = '/Users/rahulsomani/Desktop/lighting-cast.onnx',
    mode  = 'classifier',
    class_labels = list(learn.dls.vocab),
    image_input_names  = ['input_image'],
    preprocessing_args = preprocessing_args,
    minimum_ios_deployment_target = '11.2'
)

1/157: Converting Node Type Conv
2/157: Converting Node Type BatchNormalization
3/157: Converting Node Type Clip
4/157: Converting Node Type Conv
5/157: Converting Node Type BatchNormalization
6/157: Converting Node Type Clip
7/157: Converting Node Type Conv
8/157: Converting Node Type BatchNormalization
9/157: Converting Node Type Conv
10/157: Converting Node Type BatchNormalization
11/157: Converting Node Type Clip
12/157: Converting Node Type Conv
13/157: Converting Node Type BatchNormalization
14/157: Converting Node Type Clip
15/157: Converting Node Type Conv
16/157: Converting Node Type BatchNormalization
17/157: Converting Node Type Conv
18/157: Converting Node Type BatchNormalization
19/157: Converting Node Type Clip
20/157: Converting Node Type Conv
21/157: Converting Node Type BatchNormalization
22/157: Converting Node Type Clip
23/157: Converting Node Type Conv
24/157: Converting Node Type BatchNormalization
25/157: Converting Node Type Add
26/157: Converting Node Type Conv


In [ ]:
# borrowed from the CoreML Survival Guide, written by Matthijs Hollemans
def get_nn_spec(spec):
    "spec is of type `Model_pb2.Model`, accessed via coreml_model.get_spec()"
    if   spec.WhichOneof("Type") == 'neuralNetwork': return spec.neuralNetwork
    elif spec.WhichOneof("Type") == 'neuralNetworkClassifier': return spec.neuralNetworkClassifier
    elif spec.WhichOneof("Type") == 'neuralNetworkRegressor':  return spec.neuralNetworkRegressor
    return None

In [ ]:
spec = coreml_model.get_spec()
nn   = get_nn_spec(spec)

In [ ]:
# store away old layers to add back to the reconstructed network
old_layers = copy.deepcopy(nn.layers)
del nn.layers[:]

# names of inputs and outputs of the scaling layer
input_name  = old_layers[0].input[0]
output_name = f"{input_name}_scaled"

# create and add scaling layer to new network
scale_layer = nn.layers.add()
scale_layer.name = "scale_layer"
scale_layer.input.append(input_name)
scale_layer.output.append(output_name)

scale_layer.scale.scale.floatValue.extend([
    1/red_sdev, 1/green_sdev, 1/blue_sdev
])
scale_layer.scale.shapeScale.extend([3,1,1])

# add back all the old layers
nn.layers.extend(old_layers)
nn.layers[1].input[0] = output_name

In [ ]:
coreml_path = '/Users/rahulsomani/Desktop/lighting-cast.mlmodel'
coremltools.utils.save_spec(spec, coreml_path)

### CoreML Inference

In [ ]:
coreml_model = coremltools.models.MLModel(coreml_path)

In [ ]:
coreml_preds = coreml_model.predict({'input_image': open_image(f)})
pprint(coreml_preds)

{'classLabel': 'shot_lighting_cast_hard',
 'lighting-cast': {'shot_lighting_cast_hard': 0.7405732274055481,
                   'shot_lighting_cast_soft': 0.2594267427921295}}


### Compare Preds Across Frameworks

The predictions 

In [ ]:
fastai_pred = learn.predict(f)[-1]
coreml_preds = [v for k,v in coreml_preds['lighting-cast'].items()]

In [ ]:
print(f"FastAI:  {fastai_pred[0].item(), fastai_pred[1].item()}")
print(f"PyTorch: {torch_pred[0][0].item(), torch_pred[0][1].item()}")
print(f"ONNX:    {preds_onnx[0][0][0], preds_onnx[0][0][1]}")
print(f"CoreML:  {coreml_preds[0], coreml_preds[1]}")

FastAI:  (0.7402700185775757, 0.2597300112247467)
PyTorch: (0.7402703166007996, 0.25972968339920044)
ONNX:    (0.74026996, 0.25973007)
CoreML:  (0.7405732274055481, 0.2594267427921295)
